In [ ]:
%cd ..
%reload_ext autoreload
%autoreload 2

In [ ]:
import eos
import h5py
import matplotlib.pyplot as plt
import numpy as np
import pyrender
import trimesh
import pickle

from xml.dom import minidom
from collections import defaultdict

from face_reconstruction.model import BaselFaceModel
from face_reconstruction.landmarks import save_bfm_landmarks

from env import BASEL_FACE_MODEL_PATH

# 1. Load model

In [ ]:
bfm = BaselFaceModel.from_h5("model2019_face12.h5")

In [ ]:
n_shape_coefficients = bfm.get_n_shape_coefficients()
n_expression_coefficients = bfm.get_n_expression_coefficients()
n_color_coefficients = bfm.get_n_color_coefficients()

In [ ]:
face_mesh = bfm.draw_sample([0 for _ in range(n_shape_coefficients)], [0 for _ in range(n_expression_coefficients)], [0 for _ in range(n_color_coefficients)])

In [ ]:
face_trimesh = bfm.convert_to_trimesh(face_mesh)
#face_trimesh.export(f"{BASEL_FACE_MODEL_PATH}/model2019_face12.ply")

# 2. Load manually picked landmark coordinates from MeshLab file

In [ ]:
def parse_xml_point(xml_point):
    point_id = int(xml_point.attributes['name'].value)
    coordinates = np.array([float(xml_point.attributes['x'].value), float(xml_point.attributes['y'].value), float(xml_point.attributes['z'].value)])
    return point_id, coordinates

In [ ]:
xml_doc = minidom.parse(f"{BASEL_FACE_MODEL_PATH}/model2019_face12_landmarks.pp")

In [ ]:
points = xml_doc.getElementsByTagName("point")
multi_pie_landmark_coordinates = [parse_xml_point(point) for point in points]

In [ ]:
multi_pie_landmark_coordinates

# 3. Find corresponding vertex IDs in mesh

In [ ]:
closest_vertices = {}
closest_distances = {}
for vertex_id, vertex in enumerate(face_trimesh.vertices):
    for landmark_idx, landmark in multi_pie_landmark_coordinates:
        distance = np.linalg.norm(landmark - vertex)
        if landmark_idx not in closest_vertices or distance < closest_distances[landmark_idx]:
            closest_vertices[landmark_idx] = vertex_id
            closest_distances[landmark_idx] = distance

In [ ]:
closest_vertices

In [ ]:
save_bfm_landmarks(closest_vertices, "model2019_face12_landmarks")

# 4. Visualize outputs

In [ ]:
for vertex_id in closest_vertices.values():
    vertex = face_trimesh.visual.vertex_colors[vertex_id]
    vertex[0] = 255
    vertex[1] = 0
    vertex[2] = 0

In [ ]:
perspective_camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0, aspectRatio=1.414)
directional_light = pyrender.DirectionalLight(color=[1.0, 1.0, 1.0], intensity=2.0)

In [ ]:
scene = pyrender.Scene()
scene.add(pyrender.Mesh.from_trimesh(face_trimesh))
scene.add(perspective_camera, pose=np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 300], [0, 0, 0, 1]])) # Position camera just in front of face
scene.add(directional_light)

In [ ]:
pyrender.Viewer(scene, use_raymond_lighting=True)